<a href="https://colab.research.google.com/github/daviddkovacs/EBD-GPR-CNC/blob/main/Retrieval_script_EBD_GPR_CNC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap
!pip install earthengine-api
!pip install PyCRS
!pip install restee
!pip install config
!pip install geojson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyCRS: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=347a990e9b259bce3e6d9ffd522a1d5c7f027e1cc46b80c173f5d7674a00e946
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built PyCRS
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 52.6 MB/s eta 0:00:00


In [2]:
import geemap, ee, os, sys, pycrs, datetime
import geemap.colormaps as cm

from ipyleaflet import *
from ipywidgets import Label
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import config
import json
import csv
import json
import geojson
import geopandas as gpd
import rasterio
import importlib
import numpy as np

import requests
import zipfile
import os
from os import listdir
from os.path import isfile,join


In [ ]:
import restee as ree
from google.auth.transport.requests import AuthorizedSession

!earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

class EESessionContainer(ree.EESession):
    def __init__(self, project, session):
        self._PROJECT = project
        self._SESSION = session

# ADD YOUR OWN CREDENTIALS
ee_session = EESessionContainer('your credentials', session)
ee.Initialize(ee_session.session.credentials, project='your project')

W0426 08:30:09.670278 134776568561664 _default.py:683] No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
Fetching credentials using gcloud
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&state=k9qhKGu3uoP7kPnbiAE9rHVZAwS9EF&prompt=consent&token_usage=remote&access_type=offline&code_challenge=Lu252EvhWHOKSe5z5Uoz8JtHG85dNLU0eUINgPWpMXQ&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AeaYSHBqPjpSE5uXm-OMuJsFCuJ9ZDMlHY

In [ ]:
# Define your own ROI
roi= ee.Geometry.Polygon(
    [[[11.67427222, 48.29512500],
      [11.67132778, 48.23868889],
      [11.73912778, 48.23709167],
      [11.74214722, 48.29352500]]])

startDateStr = '2018-04-01'
startDateGEE = ee.Date(startDateStr)
endDateStr = '2018-09-01'
startDateGEE = ee.Date(startDateStr)
endDate = datetime.datetime.strptime(endDateStr, '%Y-%m-%d').date()

timeWindows = 5 # change timewindow to your desired compositing factor

assetPath='add your asset path'

# Desert and water masks
bare_cover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('bare-coverfraction').lte(90);
lakes = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification').eq(80)
lakemask = lakes.eq(0);

fileName='CNC_'
limitUp  = 25
limitDown = 0
variables_GREEN = {'CNC':['CNC', 25, 0.001]}
variable = 'CNC'
bands = ['GREEN', 'UNCERTAINTY_GREEN']

S2MSI = ee.ImageCollection('COPERNICUS/S2_SR')

sys.setrecursionlimit(10000000)

In [ ]:
import model_CNC #Import your model, you can drag and drop the downloaded file from GitHub here

Mounted at /content/drive


In [ ]:
print(globals()['model_CNC'])
print(globals()['model_CNC'].hyp_sig_unc_GREEN)
print(globals()['model_CNC'].XTrain_dim_GREEN)

<module 'model7_CNC' from '/content/model7_CNC.py'>
113.164778648274
318


In [ ]:
def sequence_GREEN(variable):
	sequence_GREEN = []
	model = globals()['model_' + variable]
	for i in range(0, model.XTrain_dim_GREEN):
		sequence_GREEN.append(str(i))
	return sequence_GREEN

def getInputDates(i):
  fecha_inicio = startDateGEE.advance(ee.Number(i).multiply(timeWindows),'day')
  fecha_fin = fecha_inicio.advance(timeWindows, 'day')
  #fecha_fin = endDateGEE()
  fecha_str = datetime.datetime.utcfromtimestamp(fecha_inicio.getInfo()['value']/1000.0).strftime('%Y%m%d')
  return {'fecha_inicio':fecha_inicio, 'fecha_fin':fecha_fin, 'fecha_str':fecha_str}

def maskS2cloud(image):
  model = globals()['model_CNC']
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(model.scaleFactor_GREEN).copyProperties(qa).set('system:time_start', qa.get('system:time_start'))

def addVariables(image):
  date = ee.Date(image.get("system:time_start"));
  years = date.difference(ee.Date('1970-01-01'),'days');
  return image.addBands(ee.Image(years).rename('t').int()); #rename('t') után volt .int()

In [ ]:
def calculate_GREEN(fecha_inicio, fecha_fin, variable):

  model = globals()['model_' + variable]
  image = ee.Image(S2MSI
  .filterDate(fecha_inicio, fecha_fin)
  .filterBounds(roi)
  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 40)
  .map(maskS2cloud)
  .select(model.bands)
  .max()
  .clip(roi));

  im_norm_ell2D_hypell = image.subtract(model.mx_GREEN).divide(model.sx_GREEN).multiply(model.hyp_ell_GREEN).toArray().toArray(1);
  im_norm_ell2D = image.subtract(model.mx_GREEN).divide(model.sx_GREEN).toArray().toArray(1);
  PtTPt  = im_norm_ell2D_hypell.matrixTranspose().matrixMultiply(im_norm_ell2D).arrayProject([0]).multiply(-0.5); #OK

  PtTDX  = ee.Image(model.X_train_GREEN).matrixMultiply(im_norm_ell2D_hypell).arrayProject([0]).arrayFlatten([sequence_GREEN(variable)]);
  arg1   = PtTPt.exp().multiply(model.hyp_sig_GREEN);
  k_star = PtTDX.subtract(model.XDX_pre_calc_GREEN.multiply(0.5)).exp().toArray();
  mean_pred = k_star.arrayDotProduct(model.alpha_coefficients_GREEN.toArray()).multiply(arg1);
  mean_pred = mean_pred.toArray(1).arrayProject([0]).arrayFlatten([[variable + '_GREEN']]);
  mean_pred = mean_pred.add(model.mean_model_GREEN);

  k_star_uncert = PtTDX.subtract(model.XDX_pre_calc_GREEN.multiply(0.5)).exp().multiply(arg1).toArray();
  Vvector = ee.Image(model.Linv_pre_calc_GREEN).matrixMultiply(k_star_uncert.toArray(0).toArray(1)).arrayProject([0])
  Variance = ee.Image(model.hyp_sig_unc_GREEN).toArray().subtract(Vvector.arrayDotProduct(Vvector)).abs().sqrt()
  Variance = Variance.toArray(1).arrayProject([0]).arrayFlatten([[variable + '_UNCERTAINTY_GREEN']])

  image= image.addBands(mean_pred);
  image = image.addBands(Variance);

  return image.select(variable + '_GREEN', variable + '_UNCERTAINTY_GREEN')#, variable + '_QUALITY_FLAG');

Create a function to iterate through all the images in the collection


In [ ]:
#Iterations:
def maploop():
    #startDate = datetime.datetime.strptime(startDateStr, "%Y-%m-%d").date()
    startDate = datetime.datetime.strptime(startDateStr, "%Y-%m-%d").date()
    daysIterations = abs((startDate - endDate) // timeWindows).days

    for i in range(0, daysIterations):
        print(getInputDates(i)['fecha_str'])
        imageHolder = ee.Image().set('system:time_start', startDateGEE.advance(ee.Number(i).multiply(timeWindows), 'days').millis())

        for variable_GREEN in variables_GREEN:
            params = variables_GREEN[variable_GREEN]
            variable = params[0]
            limitUp = params[1] #This is the maximum variable value defined in the first cell
            limitDown = params[2]
            imagen = calculate_GREEN(getInputDates(i)['fecha_inicio'], getInputDates(i)['fecha_fin'], variable)#, limitUp, limitDown)
            imageHolder = imageHolder.addBands(imagen)

            image_export = imageHolder.select('CNC_GREEN', 'CNC_UNCERTAINTY_GREEN')

            image_export = image_export.mask(lakemask)
            image_export = image_export.mask(bare_cover)

            data_range = limitUp-limitDown
            #image_export= image_export.multiply(255/data_range).uint8()
            image_export = image_export.set('system:time_start', startDateGEE.advance(ee.Number(i).multiply(timeWindows),'days').millis());

            exportar = ee.batch.Export.image.toAsset(
            assetId=assetPath + getInputDates(i)['fecha_str'] + variable_GREEN + fileName,
            image=image_export,
            maxPixels=4731453308,
            description=getInputDates(i)['fecha_str'] + variable_GREEN, #+ fileName,
            scale=20,
            # region=roi.geometry()
            region=roi
            )
            exportar.start()
            exportar.status()


In [ ]:
maploop()

20180401
20180402
20180403
20180404
20180405


20180406
20180407
20180408
20180409
20180410
20180411
20180412
20180413
20180414
20180415
20180416
20180417
20180418
20180419
20180420
20180421
20180422
20180423
20180424
20180425
20180426
20180427


20180428


20180429
20180430
20180501
20180502
20180503
20180504
20180505
20180506
20180507
20180508
20180509
20180510
20180511
20180512
20180513
20180514
20180515
20180516
20180517
20180518
20180519
20180520
20180521
20180522
20180523
20180524
20180525
20180526
20180527
20180528
20180529
20180530


20180531
20180601
20180602
20180603
20180604
20180605
20180606
20180607
20180608
20180609
20180610
20180611
20180612
20180613
20180614
20180615
20180616
20180617
20180618
20180619
20180620
20180621
20180622
20180623
20180624
20180625
20180626
20180627
20180628
20180629
20180630
20180701
20180702
20180703
20180704
20180705
20180706
20180707
20180708
20180709
20180710
20180711
20180712
20180713
20180714
20180715
20180716
20180717
20180718
20180719
20180720
20180721
20180722
20180723
20180724
20180725
20180726
20180727
20180728
20180729
20180730
20180731
20180801
20180802
20180803
20180804
20180805
20180806
20180807
20180808
20180809
20180810
20180811
20180812
20180813
20180814
20180815
20180816
20180817
20180818
20180819
20180820
20180821
20180822
20180823
20180824
20180825
20180826
20180827
20180828
20180829
20180830
20180831
